In [14]:
!source /mnt/nas/suehyun/.bashrc

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
!python -m pip install flash-attn --no-build-isolation

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached flash_attn-2.7.0.post2.tar.gz (2.7 MB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [12 lines of output]
      fatal: not a git repository (or any of the parent directories): .git
      
      
      torch.__version__  = 2.4.0+cu121
      
      
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-a8v5ls6d/flash-attn_46c7c3e7672b490d84cdfab674b0d347/setup.py", line 164, in <module>
          raise RuntimeError(
      RuntimeError: FlashAttention is only supported on CUDA 11.7 and above.  Note: make sure nvcc has a supported version by running nvcc -V.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating p

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import torch
# os.environ["CUDA_VISIBLE_DEVICES"] = "3"

tokenizer = AutoTokenizer.from_pretrained("suehyunpark/potpourri-8b-inst-fft-induction-bc-trajectory-max10-per-task-check-answer")
model = AutoModelForCausalLM.from_pretrained("suehyunpark/potpourri-8b-inst-fft-induction-bc-trajectory-max10-per-task-check-answer",
                                             torch_dtype=torch.bfloat16,
                                             )
model.to("cuda:3")


/home/suehyun/.conda/envs/barc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 10.20it/s]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [8]:
import json

with open("problems/arc_problems_train_400_extra_newline_v2.jsonl") as f:
    for line in f:
        data = json.loads(line)
        messages = data["messages"]
        break
    
inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

In [18]:
with torch.inference_mode():
    outputs = model.generate(inputs, max_new_tokens=1024)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


system

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a world-class puzzle solver with exceptional pattern recognition skills and expertise in Python programming. Your task is to analyze puzzles and provide Python solutions.user

Given input-output grid pairs as reference examples, carefully observe the patterns to predict the output grid for new test input. Each pair follows the same transformation rule. Grids are 2D arrays represented as strings, with cells (colors) separated by spaces and rows by newlines.
Here are the input and output grids for the reference examples:
Example 1
Input:
Black Blue Black Black Red Red Black Black Black Black Black Black Black Black Black Red Black Red Black Black Black Red
Black Blue Black Black Black Black Black Black Black Black Black Black Black Black Black Red Black Black Black Black Black Black
Black Blue Black Black Black Black Black Black Black Black Black Black Black Black Black Red Black Black Black Black Black Black


In [3]:
# ... existing imports ...
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import torch
import json
import datetime
from tqdm import tqdm
import re
from verification_tools import *
from arc import train_problems, validation_problems

os.environ["TOKENIZERS_PARALLELISM"] = "false"

def parse_code(paragraph):
    """Extracts code blocks from markdown text"""
    code_block_pattern = re.compile(r"```python(.*?)```", re.DOTALL)
    matches = code_block_pattern.findall(paragraph)
    code_blocks = [match.strip() for match in matches]
    
    if code_blocks:
        return code_blocks
    
    code_block_pattern = re.compile(r"```(.*?)```", re.DOTALL)
    matches = code_block_pattern.findall(paragraph)
    return [match.strip() for match in matches]

def get_arc_problem(uid):
    for problem in train_problems + validation_problems:
        if problem.uid == uid:
            return problem
    return None

# Load problem data
with open("problems/arc_problems_train_400_extra_newline_v2.jsonl") as f:
    data = [json.loads(line) for line in f]

# Setup inference parameters
num_samples_per_problem = 1
max_attempts = 6
temperature = 0.8
datetime_str = datetime.datetime.now().strftime("%m%d%H%M%S%f")
saving_file = f"problems/arc_problems_train_400_extra_newline_v2_{model.config._name_or_path.split('/')[-1]}_temp_{temperature}_{datetime_str}.jsonl"

all_responses = []


# Main inference loop
for d in tqdm(data, desc="Processing problems"):
    uid = d["uid"]
    problem = get_arc_problem(uid)
    messages = d["messages"]
    
    input_grids = [pair.x for pair in problem.train_pairs]
    output_grids = [pair.y for pair in problem.train_pairs]
    
    trajectories = []
    for idx in tqdm(range(num_samples_per_problem), desc="Processing trajectories"):
        chat_history = messages[:2].copy()
        trajectory_verdicts = []
        trajectory_outputs = []
        success = False
        
        num_attempts = 0
        for _ in range(max_attempts):
            num_attempts += 1
            
            # Generate response
            inputs = tokenizer.apply_chat_template(chat_history, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda:3")
            # with torch.inference_mode():
            outputs = model.generate(
                inputs, 
                max_new_tokens=1536,
                temperature=temperature,
                do_sample=True,
                num_return_sequences=1,
                stopping_criteria=[StopOnTokens()]
            )
            hypothesis = tokenizer.decode(outputs[0], skip_special_tokens=True)
            print(f"Hypothesis: {hypothesis}")
            
            chat_history.append({"role": "assistant", "content": hypothesis})
            
            try:
                code = parse_code(hypothesis)[0]
                result_str, verdicts, return_output_grids = verify_transformation(code, input_grids, output_grids)
                print("Result:", result_str)
                
                chat_history.append({"role": "user", "content": result_str})
                trajectory_verdicts.append(verdicts)
                trajectory_outputs.append(return_output_grids)
                
                if all(verdicts):
                    success = True
                    break
            except Exception as e:
                print(f"Error: {e}")
                trajectory_verdicts.append(None)
                trajectory_outputs.append(None)
                result_str = None
                break
        
        trajectories.append({
            "success": success,
            "num_attempts": num_attempts,
            "output_messages": chat_history[2:],
            "verdicts": trajectory_verdicts,
            "output_grids": trajectory_outputs,
        })
        
        if success:
            print(f"✓ Success on attempt {num_attempts}")
        elif num_attempts >= max_attempts:
            print(f"✗ Failed after {num_attempts} attempts")
    
    # Save response data
    response_data = {
        "uid": uid,
        "input_messages": messages[:2],
        "base_model": model.config._name_or_path,
        "trajectories": trajectories
    }
    all_responses.append(response_data)
    
    # Save results after each problem
    with open(saving_file, "w") as f:
        f.write("\n".join(json.dumps(p) for p in all_responses))
    break

print(f"\nInference complete. Results saved to {saving_file}")

Processing problems:   0%|          | 0/400 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Hypothesis: system

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a world-class puzzle solver with exceptional pattern recognition skills and expertise in Python programming. Your task is to analyze puzzles and provide Python solutions.user

Given input-output grid pairs as reference examples, carefully observe the patterns to predict the output grid for new test input. Each pair follows the same transformation rule. Grids are 2D arrays represented as strings, with cells (colors) separated by spaces and rows by newlines.
Here are the input and output grids for the reference examples:
Example 1
Input:
Black Blue Black Black Red Red Black Black Black Black Black Black Black Black Black Red Black Red Black Black Black Red
Black Blue Black Black Black Black Black Black Black Black Black Black Black Black Black Red Black Black Black Black Black Black
Black Blue Black Black Black Black Black Black Black Black Black Black Black Black Black Red Black Black Black Black 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Result: Result of the transform() function for every case in the reference and test examples:
Case 1: Incorrect output
Case 2: Incorrect output
Case 3: Incorrect output
Hypothesis: system

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a world-class puzzle solver with exceptional pattern recognition skills and expertise in Python programming. Your task is to analyze puzzles and provide Python solutions.user

Given input-output grid pairs as reference examples, carefully observe the patterns to predict the output grid for new test input. Each pair follows the same transformation rule. Grids are 2D arrays represented as strings, with cells (colors) separated by spaces and rows by newlines.
Here are the input and output grids for the reference examples:
Example 1
Input:
Black Blue Black Black Red Red Black Black Black Black Black Black Black Black Black Red Black Red Black Black Black Red
Black Blue Black Black Black Black Black Black Black Black Black Black Black 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Result: Result of the transform() function for every case in the reference and test examples:
Case 1: Incorrect output
Case 2: Incorrect output
Case 3: Incorrect output
Hypothesis: system

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a world-class puzzle solver with exceptional pattern recognition skills and expertise in Python programming. Your task is to analyze puzzles and provide Python solutions.user

Given input-output grid pairs as reference examples, carefully observe the patterns to predict the output grid for new test input. Each pair follows the same transformation rule. Grids are 2D arrays represented as strings, with cells (colors) separated by spaces and rows by newlines.
Here are the input and output grids for the reference examples:
Example 1
Input:
Black Blue Black Black Red Red Black Black Black Black Black Black Black Black Black Red Black Red Black Black Black Red
Black Blue Black Black Black Black Black Black Black Black Black Black Black 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Result: Result of the transform() function for every case in the reference and test examples:
Case 1: Incorrect output
Case 2: Incorrect output
Case 3: Incorrect output
Hypothesis: system

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a world-class puzzle solver with exceptional pattern recognition skills and expertise in Python programming. Your task is to analyze puzzles and provide Python solutions.user

Given input-output grid pairs as reference examples, carefully observe the patterns to predict the output grid for new test input. Each pair follows the same transformation rule. Grids are 2D arrays represented as strings, with cells (colors) separated by spaces and rows by newlines.
Here are the input and output grids for the reference examples:
Example 1
Input:
Black Blue Black Black Red Red Black Black Black Black Black Black Black Black Black Red Black Red Black Black Black Red
Black Blue Black Black Black Black Black Black Black Black Black Black Black 

Processing problems:   0%|          | 0/400 [03:16<?, ?it/s]

Result: Result of the transform() function for every case in the reference and test examples:
Case 1: Incorrect output
Case 2: Incorrect output
Case 3: Incorrect output
✗ Failed after 6 attempts

Inference complete. Results saved to problems/arc_problems_train_400_extra_newline_v2_potpourri-8b-inst-fft-induction-bc-trajectory-max10-per-task-check-answer_temp_0.8_1119180017167268.jsonl
